### ATGreen - Compute set of pre-defined indices inspired by institutional targets and load raster on database

In [1]:
# Load required packages and dictioaries
from atgreen import *

PATH=''
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
city_names_final=pickle.load(open(f"{PATH}/dicts/city_names_final.pickle", 'rb'))
city_names_final_reverse={v:k for k,v in city_names_final.items()}

###### Set precedural definition of pre-defined accessibility indices inspired by institutional targets.


In [2]:
precomputed_targets={'WHO':
                         {'index':'minimum_distance', 
                          'min_park_size':0.5, 
                          'time_threshold':5,
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network'},
                    'Berlin1':
                         {'index':'minimum_distance', 
                          'min_park_size':0.5,
                          'time_threshold':10,
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network'},
                    'Berlin2':
                         {'index':'minimum_distance', 
                          'min_park_size':10, 
                          'time_threshold':15, 
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network'},
                    'Berlin3':
                         {'index':'per_person',
                          'min_park_size':0.5, 
                          'time_threshold':15,
                          'green_type':"{parks,forests,grass}", 
                          'source': 'OSM',
                          'distances':'street-network',
                          'exposure_target':6 #exposure target expressed in meters only for per_person indices
                         },
                    'Natural_England1':
                         {'index':'minimum_distance', 
                          'min_park_size':2, 
                          'time_threshold':5,
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network'},
                    'Natural_England2':
                         {'index':'minimum_distance', 
                          'min_park_size':20, 
                          'time_threshold':25,
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network'},
                    'Italy_perperson':
                         {'index':'per_person', 
                          'min_park_size':0.5, 
                          'time_threshold':30,
                          'green_type':"{parks,forests,grass}", 
                          'source':'OSM',
                          'distances':'street-network',
                          'exposure_target':9 #exposure target expressed in meters only for per_person indices
                         },
                    'ESA_WHO_exposure':
                         {'index':'exposure', 
                          'min_park_size':0,
                          'time_threshold':5,
                          'green_type':'Any',
                          'source':'ESA',
                          'distances':'street-network',
                          'exposure_target':0.5}
                    }

### Preliminary step

### Run for all cities

Load file with raster naming convention

In [136]:
raster_band=pd.read_csv(f"{PATH}/ForWebInterface/raster/raster_band_to_label.csv")
label_to_raster={v:k for k,v in raster_band.set_index('raster_band').to_dict()['label'].items()}

cols=['x', 'y','population'] 
cols_for_extension=[[i, f'TargetSatisfied_{i}',f'BetterThan_{i}', f'Equal_{i}'] for i in ['WHO', 'Berlin1', 'Berlin2','Berlin3','Natural_England1','Natural_England2','Italy_perperson','ESA_WHO_exposure']]
for i in cols_for_extension:
    cols.extend(i)

Compute indices

In [3]:
for city in city_names_final_reverse.keys():
    for ind,i in enumerate(['WHO', 'Berlin1', 'Berlin2','Berlin3','Natural_England1','Natural_England2','Italy_perperson','ESA_WHO_exposure']):

        print(i)
        index_params=precomputed_targets[i]
        index_storage_name=i
        min_intersection=min(0.025,index_params['min_park_size'])
        index=accessibility_index_pipeline(city, index_params, index_storage_name, db_params, min_intersection)
        if ind==0:
            final=index.copy()
        else:
            final=pd.merge(final, index, on=['x','y'])
    break
final.to_csv(f"{PATH}/indices/{city}_fixedIndices_final.csv", index=False)

Berlin3


Generate raster and load into db


In [ ]:
# Compute indices
for firstcity,city in enumerate(city_names_final_reverse.keys()):
    print(firstcity)
    print(city)
    
    unreachable=pd.read_csv(f"/mnt/work/accessToGreen/data/processed_data/OSRMdistances/{city_names_final_reverse[city]}.dist.bz2")
    unreachable=unreachable[unreachable['walk_access_source']==0][['x_source', 'y_source']].drop_duplicates()
    unreachable['unreachable']=1
    
    final=pd.read_csv(f"{PATH}/indices/{city}_fixedIndices_final.csv")
    
    grid_only=query4grid_unmasked(city, db_params)
    grid_pop=query4grid(city, db_params)
    grid_only.loc[grid_only[['x','y']].apply(tuple, axis=1).isin(list(final[['x','y']].apply(tuple, axis=1)))==False, 'population']=0
    grid_only=pd.merge(grid_only[['x', 'y', 'geom']], final, how='left', on=['x', 'y'])
    grid=pd.merge(grid_only, grid_pop[grid_pop['inbound']==1][['x', 'y', 'population', 'inbound']], how='left', on=['x', 'y'])
    # Outside city boundary
    grid=grid.fillna(-9)
    grid=pd.merge(grid, unreachable, left_on=['x', 'y'], right_on=['x_source', 'y_source'], how='left')
    
    for index in ['WHO', 'Berlin1', 'Berlin2', 'Berlin3', 'Natural_England1', 'Natural_England2', 'ESA_WHO_exposure']:
        # Outside city boundary
        grid.loc[(grid['inbound']!=1), f'{index}']=-9
        grid.loc[(grid['inbound']!=1), f'TargetSatisfied_{index}']=-9
        grid.loc[(grid['inbound']!=1), f'BetterThan_{index}']=-9
        # No population
        grid.loc[(grid['population']==0) & (grid['inbound']==1), f'{index}']=-1
        grid.loc[(grid['population']==0) & (grid['inbound']==1), f'TargetSatisfied_{index}']=-1
        grid.loc[(grid['population']==0) & (grid['inbound']==1), f'BetterThan_{index}']=-1
        # Unreachable cells
        grid.loc[(grid[index]==-2) & (grid['inbound']==1) & (grid['unreachable']==1), f'{index}']=-3
        grid.loc[(grid[index]==-2) & (grid['inbound']==1) & (grid['unreachable']==1), f'TargetSatisfied_{index}']=-3
        grid.loc[(grid[index]==-2) & (grid['inbound']==1) & (grid['unreachable']==1), f'BetterThan_{index}']=-3
        #Residuals -2 hve no green within 3 km buffer
    
    #drop columns no longer needed
    grid=grid.drop(columns=['inbound', 'unreachable'])
    
    #Generate proper BetterThan and Equal
    indices=['WHO','Berlin1','Berlin2','Berlin3','Natural_England1','Natural_England2', 'Italy_perperson', 'ESA_WHO_exposure']

    # - Rename the better than into betterthanequal
    for index in indices:
        grid=grid.rename(columns={f"BetterThan_{index}": f"BetterThanEqual_{index}"})

    # - Identify equal
    for index in indices:
        print(index)
        #Check
        tmp=grid[grid[f"{index}"]>0][[f'{index}',f"BetterThanEqual_{index}"]].groupby(f"BetterThanEqual_{index}").std().reset_index()
        if len(tmp[(tmp[f'{index}']>0)])>0:
            raise Exception('Error in the computation of better than equal')
        #Compute eual to
        tmp=grid[['population',f"BetterThanEqual_{index}"]].groupby(f"BetterThanEqual_{index}").sum().reset_index()
        tmp[f"Equal_{index}"]=tmp[f"population"]/tmp[tmp[f"population"]>0][f"population"].sum()
        tmp=tmp.drop(columns=['population'])
        grid=pd.merge(grid, tmp, on=[f"BetterThanEqual_{index}"], how='left')
        grid.loc[grid[f"{index}"]==-9, f"Equal_{index}"]=-9
        grid.loc[grid[f"{index}"]==-1, f"Equal_{index}"]=-1

    # - Identify proper betterthan by subtracting equal to betterthanequal
    for index in indices:
        grid[f"BetterThan_{index}"]=grid[f"BetterThanEqual_{index}"]-grid[f"Equal_{index}"]

    del [grid_pop, grid_only]

    grid['geom']=grid['geom'].centroid

    points = list(zip(np.round(grid.geometry.x,6),np.round(grid.geometry.y,6)))
    xRange=np.round(grid.geometry.x,6).unique()
    yRange=np.round(grid.geometry.y,6).unique()

    if len(xRange)!=grid['x'].max() or len(yRange)!=grid['y'].max() :
        raise Exception('Error in raster dimensions')

    #define raster resolution
    xRes = (np.round(grid.geometry.x,6).max()-np.round(grid.geometry.x,6).min())/(len(xRange)-1)
    yRes = (np.round(grid.geometry.y,6).max()-np.round(grid.geometry.y,6).min())/(len(yRange)-1)

    #definition of the raster transform array
    from rasterio.transform import Affine
    transform = rasterio.transform.from_bounds(min(xRange)-(xRes/2), min(yRange)-(yRes/2), max(xRange)+(xRes/2), max(yRange)+(yRes/2), len(xRange), len(yRange))


    #get crs as wkt
    from rasterio.crs import CRS
    rasterCrs = CRS.from_epsg(4326)
    rasterCrs.data

    #definition, register and close of interpolated raster
    interpRaster = rasterio.open(f"{PATH}/ForWebInterface/raster/{city}.tiff",
                                'w',
                                driver='GTiff',
                                height=len(yRange),
                                width=len(xRange),
                                count=len(cols),
                                dtype=rasterio.dtypes.float64,
                                crs=rasterCrs,
                                transform=transform,
                                nodata=-9)
                             
    for col in cols:
        grid[col]=np.round(grid[col].values,3)
        mat = np.array(list(grid.sort_values(by=['x','y'], ascending=True)[col]))
        mat = np.squeeze(mat)  
        mat = mat.reshape(len(xRange),len(yRange)).T #Because reshape orders by row instead of columns
        interpRaster.write(mat,label_to_raster[col])
    
    interpRaster.close()

    #Generate to SQL table
    if firstcity==0:
        mode="d"
    else:
        mode="a"
    
    # Load raster into database
    rast2sql(f"{PATH}ForWebInterface/raster/", f"{city}" ,"/mnt/work/projects/esa", "indices", mode, (len(xRange),len(yRange)), 'tiff', '4326')
    sqlRasterTable2db('indices','/mnt/work/projects/esa',db_params)

    #Remove local file
    os.remove(f"{PATH}/ForWebInterface/raster/{city}.tiff")